## LangChain with time tool
> **Expected Output**: Action: today's date

LangChain agent with time tool.  The tool is defined with a LangChain decorator and registered with the initialized agent.


In [6]:
import os
import requests
import json
import httpx
from openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from datetime import date
import warnings
from cfenv import AppEnv
import sys, os

# go one level up from cflangchainfolder/ to project root
sys.path.append(os.path.abspath(".."))
from cfutils import CFGenAIService

In [11]:
# Load CF environment
env = AppEnv()

# load your service details replace name with your Gen AI service.  Gen AI service is bound to the app
genai = CFGenAIService("gpt-oss:120b")
print("API Base:", genai.api_base)
print("Config URL:", genai.config_url)

# List available models
models = genai.list_models()
for m in models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

# construct chat_credentials
chat_credentials = {
    "api_base": genai.api_base,
    "api_key": genai.api_key,
    "model_name": models[0]["name"]
}

# 2. HTTP client (optional but recommended for custom config)
httpx_client = httpx.Client(verify=False)  # verify=False if your endpoint needs --insecure

# 3. Initialize the LLM
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],   # model name from CF service
    base_url=chat_credentials["api_base"],  # OpenAI-compatible endpoint
    api_key=chat_credentials["api_key"],    # Bearer token
    http_client=httpx_client
)

API Base: https://genai-proxy.sys.tas-ndc.kuhn-labs.com/tanzu-gpt-oss-120b-b83126a/openai
Config URL: https://genai-proxy.sys.tas-ndc.kuhn-labs.com/tanzu-gpt-oss-120b-b83126a/config/v1/endpoint
- gpt-oss:120b (capabilities: CHAT, TOOLS)


/home/vcap/app/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-proxy.sys.tas-ndc.kuhn-labs.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [12]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())


# list of tools for agent
tools = [time]

In [13]:
# initialize agent with tools and llm
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")




> Entering new AgentExecutor chain...
Could not parse LLM output: 
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:Question: whats the date today?

Thought: I need to get today's date using the time tool.

Action:
```
{
  "action": "time",
  "action_input": ""
}
```
Observation: 2025-09-19
Thought:Question: whats the date today?
Thought: I need to get today's date using the time tool.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-09-19
Thought:Question: whats the date today?
Thought: I need to retrieve today's date using the time tool.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-09-19
Thought:Question: whats the date today?
Thought: I have obtained the date using the time tool.
Final Answer: Today's date is 2025-09-19.

> Finished chain.
